# 初始設定 (每次都要執行)

In [ ]:
!pip install pycoingecko

In [ ]:
# Import packages
from pycoingecko import CoinGeckoAPI
import pandas as pd
import datetime

# CoinGecko API client
cg = CoinGeckoAPI()

# Set the dataframe
column = pd.DataFrame(columns=['項目名稱', '市值排名', '代幣價格', '24HR', '30日'])

# 項目名稱設定
- 可以建好多個字典，如「公鏈」、「SBF 全家桶」、「波卡生態」、「NFT 系列」等

In [ ]:
# Set the token list
token_list = ['ethereum', 'binancecoin', 'cardano', 'polkadot', 'solana', 'cosmos', 'avalanche-2', 'algorand', 'fantom', 'near']

# 計算價格
- 初始設定為 24HR 與 30日 的漲幅變化，可調整 `n_day` 來設定

In [ ]:
n_day = 30
start = datetime.timedelta(days=n_day)
day_1= (datetime.datetime.now() - start).strftime("%d-%m-%Y") #first-day price
day_last_1 = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%d-%m-%Y") #last-one-day price
day_last = datetime.datetime.now().strftime("%d-%m-%Y") #last-day price

for i in range(len(token_list)):
  values = list()
  values.append(token_list[i])
  day_last_price = cg.get_coin_history_by_id(id=token_list[i], date=day_last)['market_data']['current_price']['usd']
  day_last1_price = cg.get_coin_history_by_id(id=token_list[i], date=day_last_1)['market_data']['current_price']['usd']
  day_1_price = cg.get_coin_history_by_id(id=token_list[i], date=day_1)['market_data']['current_price']['usd']
  pnl_1 = (day_last_price - day_last1_price)/day_last1_price * 100
  pnl_long = (day_last_price - day_1_price)/day_1_price * 100
  market_cap_rank = cg.get_coin_by_id(token_list[i])['market_cap_rank']
  values.append(market_cap_rank)
  values.append(round(day_last_price, 2))
  values.append(round(pnl_1, 2))
  values.append(round(pnl_long, 2))
  column.loc[i] = values

column.sort_values(by='30日', ascending=False)

,項目名稱,市值排名,代幣價格,24HR,30日
4,solana,16,42.70,-7.35,77.76
1,binancecoin,3,608.63,-9.85,74.37
8,fantom,70,0.65,-8.19,59.70
0,ethereum,2,3245.66,-5.65,56.21
2,cardano,7,1.28,-5.97,7.99
6,avalanche-2,44,30.48,-15.39,4.17
7,algorand,45,1.29,-10.56,-1.53
5,cosmos,32,21.07,-8.53,-2.48
9,near,75,4.80,-12.30,-20.55
3,polkadot,8,34.66,-7.10,-21.65


# 將結果寫成 CSV 檔
- 將上方表格存成 csv 檔，可於左側檔案區按右鍵下載

In [ ]:
column.to_csv("public_chain_pnl.csv")